<a href="https://colab.research.google.com/github/maya-papaya/Analyzing-Protests-Through-Tweets/blob/main/Step%202%3A%20Fine-tuning%20Sentiment%20Analysis%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 2: Fine-tuning Sentiment Analysis Model

Next, we'll have to fine-tune a sentiment analysis model. This model should be able to extract the sentiment (positive, negative, or neutral) from a tweet.

In [ ]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 22.0 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.9 MB/s eta 0:00:00


I decided to use the [tweet_eval](https://huggingface.co/datasets/tweet_eval/viewer/sentiment/train) dataset from the HuggingFace Hub.



In [ ]:
from datasets import load_dataset

dataset = load_dataset("tweet_eval", 'sentiment')

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45615 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12284 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
train = dataset["train"]
eval = dataset["validation"]
test = dataset["test"]

Here's what the dataset looks like:

In [ ]:
train = train.rename_column('label', 'labels')
print(train)
print(train[0])

Dataset({
    features: ['text', 'labels'],
    num_rows: 45615
})
{'text': '"QT @user In the original draft of the 7th book, Remus Lupin survived the Battle of Hogwarts. #HappyBirthdayRemusLupin"', 'labels': 2}


I used the bert-base-uncased tokenizer.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
def tokenize(example):
  return tokenizer(example["text"], truncation=True, padding=True)

train_tokenized = train.map(tokenize, batched=True)
eval_tokenized = eval.map(tokenize, batched=True)
test_tokenized = test.map(tokenize, batched=True)

Map:   0%|          | 0/45615 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/12284 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Since I decided to fine-tune a bert-base-uncased model, I had to change the number of labels to 3 in the config.

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoConfig, TrainingArguments

model_name = 'bert-base-uncased'
config = AutoConfig.from_pretrained(model_name, num_labels=3)
model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="tweet-sentiment-analyzer",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import Trainer
from sklearn.metrics import accuracy_score

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=eval_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=lambda pred: {"accuracy": accuracy_score(pred.label_ids, pred.predictions.argmax(-1))},
)

In [ ]:
# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.632100,0.604597,0.742500
2,0.437800,0.661517,0.751000


TrainOutput(global_step=11404, training_loss=0.5540519414638813, metrics={'train_runtime': 662.9436, 'train_samples_per_second': 137.614, 'train_steps_per_second': 17.202, 'total_flos': 4241182222524366.0, 'train_loss': 0.5540519414638813, 'epoch': 2.0})

In [ ]:
# Evaluate the model on the test set
eval_results = trainer.evaluate(test_tokenized)

# Print the accuracy
print(f"Test Accuracy: {eval_results['eval_accuracy']}")

Test Accuracy: 0.6982253337675024


Now, I'll push the fine-tuned model to the HuggingFace Hub, so we can access it in Step 3.

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model_name = "Sentiment-Analyzer"

model.push_to_hub(model_name,
                  commit_message="Tweet Sentiment Analyzer Model",
                  private=False)

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mayapapaya/Sentiment-Analyzer/commit/c1631d61282c480188bd68c5c4d63b7bf5fab088', commit_message='Tweet Sentiment Analyzer Model', commit_description='', oid='c1631d61282c480188bd68c5c4d63b7bf5fab088', pr_url=None, pr_revision=None, pr_num=None)

Great! Now, we have our sentiment analysis model.